![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
import datetime

from QuantConnect import Resolution, Market
from QuantConnect.Algorithm import QCAlgorithm
from QuantConnect.Brokerages import BrokerageName
import pandas as pd
from QuantConnect.Orders import OrderType, OrderStatus

from algo import get_values


class WilliamsEMA(QCAlgorithm):
    def Initialize(self):
        self.RISK = 2.
        self.margin = 50.
        self.pct_of_account = 0.8
        # self.max_risk_rate = 0.02
        self.time_limit = 120
        self.max_stop_loss = 0.0020
        self.stoch_rsi_max = 0.8
        self.stoch_rsi_min = 0.2


        self.SetStartDate(2021, 1, 15)  # Set Start Date
        # self.SetEndDate(2020, 1, 10)  # Set End Date
        self.SetCash(1000)  # Set Strategy Cash
        self.SetBrokerageModel(BrokerageName.OandaBrokerage)
        # self.UniverseSettings.Resolution = Resolution.Minute
        self.SetWarmup(200)
        # self.symbol_names = ["USDAUD", "USDCAD", "USDCHF", "USDEUR", "USDGBP",
        #                      "USDHKD", "USDDKK", "USDCZK", "USDZAR",
        #                      "USDSEK", "USDSAR", "USDNOK", "USDMXN"]
        # self.symbol_names = ["AUDCAD", "EURUSD", "USDCAD", "USDCHF", "NZDUSD", "GBPUSD"]
        self.symbol_names = ['AUDCAD']

        self.symbols = []
        self.data_second = {}
        self.df = {}
        for symbol in self.symbol_names:
            self.data_second[symbol] = []
            self.symbols.append(self.AddForex(symbol, Resolution.Second, Market.Oanda, leverage=self.margin,
                                              fillDataForward=False))
            self.df[symbol] = pd.DataFrame()
            # self.df[symbol].index.name = 'datetime'
        self.order_values = {}
        self.order_types = ['stop', 'price_target']
        self._tickets = {t: {} for t in self.order_types}
        self.last_time = datetime.time(0)

    def OnOrderEvent(self, orderEvent):
        order = self.Transactions.GetOrderById(orderEvent.OrderId)
        s = order.Symbol.Value
        if order.Type == OrderType.Market and order.Status == OrderStatus.Filled:
            quantity = order.Quantity
            if quantity != 0 and len(self.order_values) > 0:
                stop = self.order_values[s]['stop']
                price_target = self.order_values[s]['price_target']
                self.Debug(f"{order.Time}, {stop}, {price_target}, {order.Price}")
                del self.order_values[s]
                if stop == price_target or not min(stop, price_target) < order.Price < max(stop, price_target):
                    self.liquidate_and_cancel_orders()
                else:
                    self._tickets['stop'][s] = self.StopMarketOrder(s, -quantity, stop, "STOP Set")
                    self._tickets['price_target'][s] = self.LimitOrder(s, -quantity, price_target, "Price Target Set")
        elif order.Type == OrderType.Limit and order.Status == OrderStatus.Filled:
            self._tickets['stop'][s].Cancel()

            del self._tickets['stop'][s]
            del self._tickets['price_target'][s]
        elif order.Type == OrderType.StopMarket and order.Status == OrderStatus.Filled:
            self._tickets['price_target'][s].Cancel()

            del self._tickets['stop'][s]
            del self._tickets['price_target'][s]

    def liquidate_and_cancel_orders(self):
        openOrders = self.Transactions.GetOpenOrders()
        if len(openOrders) > 0:
            for x in openOrders:
                self.Transactions.CancelOrder(x.Id)
        self._tickets = {t: {} for t in self.order_types}
        self.order_values = {}
        self.Liquidate()

    def OnData(self, data):
        """OnData event is the primary entry point for your algorithm. Each new data point will be pumped in here.
            Arguments:
                data: Slice object keyed by symbol containing the stock data
        """
        for symbol in data.keys():
            if symbol not in data.QuoteBars:
                continue
            quoteBarsBid = data.QuoteBars[symbol].Bid
            quoteBarsAsk = data.QuoteBars[symbol].Ask
            self.data_second[symbol.Value].append({
                'close_bid': quoteBarsBid.Close, 'open_bid': quoteBarsBid.Close,
                'high_bid': quoteBarsBid.Close, 'low_bid': quoteBarsBid.Close,
                'close_ask': quoteBarsAsk.Close, 'open_ask': quoteBarsAsk.Close,
                'high_ask': quoteBarsAsk.Close, 'low_ask': quoteBarsAsk.Close
            })

        time = self.UtcTime.replace(tzinfo=None, microsecond=0)
        if time.second != 0 and self.last_time.second < time.second:
            self.last_time = time
            return
        self.last_time = time

        new_row = {}
        # date = pd.Timestamp(data[list(data.keys())[0]].EndTime, tzinfo=datetime.timezone.utc)
        date = time
        for symbol, data_s in self.data_second.items():
            if len(data_s) == 0:
                self.Debug('skip')
                continue
            data_s = pd.DataFrame(data_s)
            new_row[symbol] = pd.Series({
                'close_bid': data_s['close_bid'].iloc[-1], 'open_bid': data_s['open_bid'].iloc[0],
                'high_bid': data_s['high_bid'].max(), 'low_bid': data_s['low_bid'].min(),
                'close_ask': data_s['close_ask'].iloc[-1], 'open_ask': data_s['open_ask'].iloc[0],
                'high_ask': data_s['high_ask'].max(), 'low_ask': data_s['low_ask'].min(),
            })
            new_df = pd.DataFrame(new_row[symbol]).transpose()
            new_df.index = [date]
            self.df[symbol] = pd.concat([self.df[symbol], new_df]).iloc[-200:]
            self.data_second[symbol] = []

        if not (datetime.time(12) < time.time() < datetime.time(20)):
            if self.Portfolio.Invested:
                self.liquidate_and_cancel_orders()
            return

        # time limit
        if 'stop' in self._tickets and len(self._tickets['stop']) > 0:
            symbols = list(self._tickets['stop'].keys())
            for s in symbols:
                time_diff = (time - self._tickets['stop'][s].Time.replace(tzinfo=None)).seconds / 60
                if self._tickets['stop'][s].Status != OrderStatus.Filled and time_diff > self.time_limit:
                    self.Debug(f"CANCEL --- {s}, {time_diff}")
                    self.liquidate_and_cancel_orders()

        if not self.IsWarmingUp and not self.Portfolio.Invested:
            self.ProfitTargets = {}
            self.StopTargets = {}
            for symbol in data.keys():
                if self.df[symbol.Value].shape[0] < 100:
                    continue

                row = new_row[symbol.Value].append(pd.Series(get_values(self.df[symbol.Value])))
                if row['spread'] > 0.0002:
                    continue
                if abs(self.df[symbol.Value]['close_bid'].pct_change(90).dropna().mean()) < 0.0001:
                    continue

                if row['fractal'] == -1 and row['ema1'] > row['ema2'] > row['ema3'] and row['ema1'] > self.df[symbol.Value].iloc[-3]['low_ask'] > row['ema3']:
                    if self.df[symbol.Value].iloc[-3]['low_ask'] < row['ema2']:
                        ema = row['ema3']
                        close = row['close_ask']
                    elif self.df[symbol.Value].iloc[-3]['low_ask'] < row['ema1']:
                        ema = row['ema2']
                        close = row['close_ask']
                    else:
                        continue
                elif row['fractal'] == 1 and row['ema1'] < row['ema2'] < row['ema3'] and row['ema1'] < self.df[symbol.Value].iloc[-3]['high_bid'] < row['ema3']:
                    if self.df[symbol.Value].iloc[-3]['high_bid'] > row['ema2']:
                        ema = row['ema2']
                        close = row['close_bid']
                    elif self.df[symbol.Value].iloc[-3]['high_bid'] > row['ema1']:
                        ema = row['ema3']
                        close = row['close_bid']
                    else:
                        continue
                else:
                    continue

                # risk_rate = (1 - (row['close_bid'] - ema) / row['close_bid']) * self.pct_of_account * self.margin
                # if abs(risk_rate) > self.max_risk_rate:
                #     ema = self.max_risk_rate / (self.pct_of_account * self.margin) - (1 - row['close_bid'])
                Stoploss = round(-(close - ema), 5)
                direction = -1 if Stoploss < 0 else 1
                Profit = round(-Stoploss * self.RISK, 5)
                Stoploss = max(Stoploss, -self.max_stop_loss) if direction == -1 else min(Stoploss, self.max_stop_loss)
                if abs(Stoploss) != abs(Profit) and abs(Profit) > max(row['spread'], 0.0005):
                    self.order_values[symbol.Value] = {'stop': close + Stoploss,
                                                       'price_target': close + Profit,
                                                       'close': close}
                else:
                    continue
                # self.Debug(f"ORDER --- {time}, {self.order_values[symbol.Value]}, {row['fractal']}, {row['d']}, {row['k']}, {row['ema1']}")
                # self.Debug(self.df[symbol.Value].index[-1])
                # intv = pd.date_range(self.df[symbol.Value].index[-100], self.df[symbol.Value].index[-1], freq='1Min', closed='left')
                # missing = [i.replace(tzinfo=None, microsecond=0).time() for i in intv if i.replace(tzinfo=None, microsecond=0) not in self.df[symbol.Value].iloc[-100:].index]
                # self.Debug(len(missing))
                # self.Debug(missing)
                # self.Debug([i.time() for i in self.df[symbol.Value].iloc[-100:].index])

            max_profit = 0
            invest_symbol = None
            direction = None
            for symbol in self.order_values.keys():
                profit = self.order_values[symbol]['price_target'] - self.order_values[symbol]['close']
                if abs(profit) > max_profit:
                    max_profit = abs(profit)
                    invest_symbol = symbol
                    direction = -1 if profit < 0 else 1
                else:
                    self.Debug(f"fail --- {symbol}, {profit}, {max_profit}")

            if invest_symbol is not None:
                self.Debug(invest_symbol)
                self.Debug(self.order_values[invest_symbol])
                self.SetHoldings(invest_symbol, direction * self.pct_of_account * self.margin)


In [2]:
w = WilliamsEMA()

In [3]:
import zipfile

In [6]:
data = zipfile.ZipFile('data/forex/oanda/second/eurusd/20140506_quote.zip')

In [5]:
data.read

Init signature: zipfile.ZipFile(file, mode='r', compression=0, allowZip64=True)
Docstring:     
Class with methods to open, read, write, close, list zip files.

z = ZipFile(file, mode="r", compression=ZIP_STORED, allowZip64=True)

file: Either the path to the file, or a file-like object.
      If it is a path, the file will be opened and closed by ZipFile.
mode: The mode can be either read 'r', write 'w', exclusive create 'x',
      or append 'a'.
compression: ZIP_STORED (no compression), ZIP_DEFLATED (requires zlib),
             ZIP_BZIP2 (requires bz2) or ZIP_LZMA (requires lzma).
allowZip64: if True ZipFile will create files with ZIP64 extensions when
            needed, otherwise it will raise an exception when this would
            be necessary.
Init docstring:
Open the ZIP file with mode read 'r', write 'w', exclusive create 'x',
or append 'a'.
File:           /opt/miniconda3/lib/python3.6/zipfile.py
Type:           type
Subclasses:     PyZipFile, BytesZipFile
